# Importing Modules

In [18]:
import numpy as np 

In [19]:
import tensorflow as tf

In [20]:
from keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing 

In [21]:
#Training Image Processing

In [22]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
                'training_set', target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 3458 images belonging to 5 classes.


In [23]:
# Test Image processing

In [24]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 859 images belonging to 5 classes.


# Building Model

In [27]:
cnn = tf.keras.models.Sequential()

In [28]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [29]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [30]:
cnn.add(tf.keras.layers.Dropout(0.5)) #just like another layer

In [31]:
# flattening
cnn.add(tf.keras.layers.Flatten())


In [32]:
#hidden layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #128 neurons

In [33]:
#output layer
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax')) 

# TRAINING THE MODEL

In [34]:
cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
cnn.fit(x=training_set, validation_data= test_set, epochs=10)  #epochs === loops 

Epoch 1/10
109/109 [==============================] - 400s 4s/step - loss: 1.3257 - accuracy: 0.4286 - val_loss: 1.1078 - val_accuracy: 0.5471
Epoch 2/10
109/109 [==============================] - 292s 3s/step - loss: 1.0976 - accuracy: 0.5599 - val_loss: 1.1051 - val_accuracy: 0.5530
Epoch 3/10
109/109 [==============================] - 292s 3s/step - loss: 1.0044 - accuracy: 0.6105 - val_loss: 1.1556 - val_accuracy: 0.5937
Epoch 4/10
109/109 [==============================] - 294s 3s/step - loss: 0.9306 - accuracy: 0.6397 - val_loss: 1.0672 - val_accuracy: 0.5867
Epoch 5/10
109/109 [==============================] - 292s 3s/step - loss: 0.8702 - accuracy: 0.6666 - val_loss: 0.9175 - val_accuracy: 0.6484
Epoch 6/10
109/109 [==============================] - 290s 3s/step - loss: 0.8409 - accuracy: 0.6729 - val_loss: 1.0026 - val_accuracy: 0.6345
Epoch 7/10
109/109 [==============================] - 266s 2s/step - loss: 0.7925 - accuracy: 0.6952 - val_loss: 0.8700 - val_accuracy: 0.6659

## preprocess new images for prediction

In [22]:
# from keras.preprocessing import image
# test_image = tf.keras.utils.load_img('Prediction/t.jpg', target_size=(64, 64))
# test_image = tf.keras.utils.img_to_array(test_image)
# test_image = np.expand_dims(test_image, axis=0)
# result = cnn.predict(test_image)

In [23]:
# training_set.class_indices

In [24]:
# if result[0][0]==1:
#     print('Daisy')
# elif result[0][1]==1:
#     print('Dandelion')
# elif result[0][2]==1:
#     print('Rose')
# elif result[0][3]==1:
#     print('SunFlower')
# elif result[0][4]==1:
#     print("Tulip")

In [25]:
# print(result)

# Saving The Model

In [36]:
tf.keras.models.save_model(cnn,'my_model2.hdf5')

# Creating the Application

In [37]:
%%writefile app.py
import streamlit as st 
from PIL import Image
import tensorflow as tf 
from image_classifier import process_image, prediction_result
import time

st.set_option('deprecation.showfileUploaderEncoding', False)

st.title("Flower Classifier")

st.write("This app can predict flowers from five categories: Daisy, Rose, Sunflower, Tulip and Dandelion")
st.write("Disclaimer: May not always give correct prediction!")
st.write("Made by: Rohan Gupta")

img = st.file_uploader("Please upload Image", type=["jpeg", "jpg", "png"])

# Display Image
st.write("Uploaded Image")
try:
    img = Image.open(img)
    st.image(img) # display the image
    img = process_image(img)


    # Prediction
    model = tf.keras.models.load_model("my_model2.hdf5")
    prediction = prediction_result(model, img)


    # Progress Bar
    my_bar = st.progress(0)
    for percent_complete in range(100):
        time.sleep(0.05)
        my_bar.progress(percent_complete + 1)

    # Output
    st.write("# Flower Type: {}".format(prediction["class"]))
    st.write("With Accuracy:", prediction["accuracy"],"%")
except AttributeError:
    st.write("No Image Selected")

Overwriting app.py


# Pre-processing

In [39]:


%%writefile image_classifier.py
import tensorflow as tf 
import numpy as np 
from PIL import Image, ImageOps	
import cv2


def process_image(img, img_size=(64, 64)):
  """
  This function is used to pre-process any chosen picture by the user
  to the appropriate format that the model accepts.
  Parameters:
    img: The input Image, opened using the PIL library
    img_size: Defaults to (64, 64) because it is the size that the 
              model accepts
  Returns:
    An Image array that is ready to be fed into the model.
  """
  # reshapes the image
  image = ImageOps.fit(img, img_size, Image.ANTIALIAS)
  # converts the image into numpy array
  image = np.asarray(image)
  # converts image from BGR color space to RGB
  img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  img_resize = (cv2.resize(img, dsize=img_size, interpolation=cv2.INTER_CUBIC))/255.

  img_reshape = img_resize[np.newaxis,...]
  
  return img_reshape


def prediction_result(model, image_data):
  """
  The function that returns the prediction result from the model
  Parameters:
    model: The model to be used to classify
    image_data: Image array that is returned by the process_image function
  
  Returns --> Dictionary with class and accuracy values
  """
  # Mapping prediction results to the Flower type
  classes = {0: "Daisy", 
             1: "Dandelion",
             2: "Rose",
             3: "Sunflower",
             4: "Tulip"}
  
  pred = model.predict(image_data)
  pred = pred.round(2)
  result = np.argmax(pred)
  
  prediction = {"class": classes[result],
                "accuracy": np.round(np.max(pred) * 100, 2)}
  
  return prediction

Overwriting image_classifier.py
